
# Clothes Image Classification  
## Logistic Regression & MLP (PyTorch)

### Objectives
1. Predict clothing types using Logistic Regression
2. Use 80:20 stratified split
3. Evaluate using Macro F1-score
4. Design MLP with 1, 2, 3 hidden layers
5. Compare ReLU, Sigmoid, and Tanh activations


## 1. Imports

In [1]:

import os
import numpy as np
from PIL import Image
from tqdm import tqdm

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader


## 2. Load Dataset

In [2]:
import kagglehub

# Download latest version from Kaggle
path = kagglehub.dataset_download("ryanbadai/clothes-dataset")
print("Path to dataset files:", path)


DATA_DIR = path
IMAGE_SIZE = (64, 64)

images = []
labels = []

base_classes_dir = os.path.join(DATA_DIR, "Clothes_Dataset")
classes = sorted(os.listdir(base_classes_dir))

for cls in classes:
    cls_path = os.path.join(base_classes_dir, cls)

    if os.path.isdir(cls_path):
        for img_name in tqdm(os.listdir(cls_path)):
            img_path = os.path.join(cls_path, img_name)

            if os.path.isfile(img_path):
                img = Image.open(img_path).convert("RGB").resize(IMAGE_SIZE)
                images.append(np.array(img))
                labels.append(cls)

images = np.array(images)
labels = np.array(labels)

print("Images shape:", images.shape)
print("Labels shape:", labels.shape)


100%|██████████| 1.37G/1.37G [01:02<00:00, 23.5MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/ryanbadai/clothes-dataset/versions/1


100%|██████████| 500/500 [00:07<00:00, 64.55it/s]


Images shape: (7500, 64, 64, 3)
Labels shape: (7500,)


## 3. Logistic Regression

In [3]:

X = images.reshape(len(images), -1)
X = X / 255.0

X_train, X_test, y_train, y_test = train_test_split(
    X, labels, test_size=0.2, stratify=labels, random_state=42
)

log_reg = LogisticRegression(
    max_iter=200,
    solver='lbfgs',
    n_jobs=-1
)

log_reg.fit(X_train, y_train)

y_pred = log_reg.predict(X_test)
f1_lr = f1_score(y_test, y_pred, average='macro')

print("Logistic Regression Macro F1:", f1_lr)


Logistic Regression Macro F1: 0.31993924951945485


## 4. PyTorch Dataset

In [4]:

class ClothesDataset(Dataset):
    def __init__(self, images, labels):
        self.images = images
        self.label_map = {cls: i for i, cls in enumerate(sorted(set(labels)))}
        self.labels = [self.label_map[l] for l in labels]

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img = torch.tensor(self.images[idx] / 255.0, dtype=torch.float32)
        img = img.reshape(-1)
        label = self.labels[idx]
        return img, label


## 5. DataLoaders

In [5]:

train_imgs, test_imgs, train_lbls, test_lbls = train_test_split(
    images, labels, test_size=0.2, stratify=labels, random_state=42
)

train_ds = ClothesDataset(train_imgs, train_lbls)
test_ds = ClothesDataset(test_imgs, test_lbls)

train_loader = DataLoader(train_ds, batch_size=64, shuffle=True)
test_loader = DataLoader(test_ds, batch_size=64, shuffle=False)

input_dim = IMAGE_SIZE[0] * IMAGE_SIZE[1] * 3
num_classes = len(set(labels))


## 6. MLP Model

In [6]:

class MLP(nn.Module):
    def __init__(self, input_dim, hidden_layers, activation, num_classes):
        super().__init__()
        layers = []
        prev = input_dim

        for h in hidden_layers:
            layers.append(nn.Linear(prev, h))
            if activation == 'relu':
                layers.append(nn.ReLU())
            elif activation == 'sigmoid':
                layers.append(nn.Sigmoid())
            elif activation == 'tanh':
                layers.append(nn.Tanh())
            prev = h

        layers.append(nn.Linear(prev, num_classes))
        self.net = nn.Sequential(*layers)

    def forward(self, x):
        return self.net(x)


## 7. Training & Evaluation

In [7]:

def train_eval(model, train_loader, test_loader, epochs=15):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=1e-3)

    for _ in range(epochs):
        model.train()
        for X, y in train_loader:
            optimizer.zero_grad()
            loss = criterion(model(X), y)
            loss.backward()
            optimizer.step()

    model.eval()
    preds, trues = [], []
    with torch.no_grad():
        for X, y in test_loader:
            preds.extend(model(X).argmax(1).numpy())
            trues.extend(y.numpy())

    return f1_score(trues, preds, average='macro')


## 8. Experiments

In [8]:

configs = {
    '1 Hidden Layer': [256],
    '2 Hidden Layers': [256, 128],
    '3 Hidden Layers': [256, 128, 64]
}

activations = ['relu', 'sigmoid', 'tanh']
results = {}

for name, cfg in configs.items():
    for act in activations:
        model = MLP(input_dim, cfg, act, num_classes)
        f1 = train_eval(model, train_loader, test_loader)
        results[(name, act)] = round(f1, 4)
        print(name, act, "Macro F1:", round(f1, 4))

results


1 Hidden Layer relu Macro F1: 0.3242
1 Hidden Layer sigmoid Macro F1: 0.1654
1 Hidden Layer tanh Macro F1: 0.0354
2 Hidden Layers relu Macro F1: 0.3696
2 Hidden Layers sigmoid Macro F1: 0.1244
2 Hidden Layers tanh Macro F1: 0.0201
3 Hidden Layers relu Macro F1: 0.34
3 Hidden Layers sigmoid Macro F1: 0.0777
3 Hidden Layers tanh Macro F1: 0.0302


{('1 Hidden Layer', 'relu'): 0.3242,
 ('1 Hidden Layer', 'sigmoid'): 0.1654,
 ('1 Hidden Layer', 'tanh'): 0.0354,
 ('2 Hidden Layers', 'relu'): 0.3696,
 ('2 Hidden Layers', 'sigmoid'): 0.1244,
 ('2 Hidden Layers', 'tanh'): 0.0201,
 ('3 Hidden Layers', 'relu'): 0.34,
 ('3 Hidden Layers', 'sigmoid'): 0.0777,
 ('3 Hidden Layers', 'tanh'): 0.0302}


## 9. Conclusion

- Logistic Regression provides a linear baseline
- Increasing hidden layers improves MLP performance
- ReLU activation performs best
- Sigmoid performs worst due to vanishing gradients
